In [1]:
include("src/NursesScheduling.jl")
using .NurseSchedules
using Logging

logger = ConsoleLogger(stderr, Logging.Debug)
global_logger(logger) 

Base.CoreLogging.SimpleLogger(IJulia.IJuliaStdio{Base.PipeEndpoint}(IOContext(Base.PipeEndpoint(RawFD(0x0000002f) open, 0 bytes waiting))), Info, Dict{Any,Int64}())

In [2]:
schedule = Schedule("./schedules/schedule_2016_august_medium.json")

[ Info: Schedule loaded!


Schedule(Dict{String,Any}("shifts" => Dict{String,Any}("nurse_2" => Any["W", "DN", "W", "W", "W", "W", "W"],"nurse_3" => Any["D", "N", "W", "L4", "U", "N", "U"],"nurse_1" => Any["R", "R", "W", "W", "D", "DN", "W"]),"schedule_info" => Dict{String,Any}("UUID" => "","month_number" => 8,"year" => 2016),"month_info" => Dict{String,Any}("children_number" => Any[3, 3, 3, 3, 3, 3, 3],"days_of_week" => Any["MO", "TU", "WE", "TH", "FR", "SA", "SU"],"day_numbers" => Any[1, 2, 3, 4, 5, 6, 7]),"employee_info" => Dict{String,Any}("time" => Dict{String,Any}("nurse_2" => "FULL","nurse_3" => "FULL","nurse_1" => "FULL"))))

In [3]:
workers, shifts = get_shifts(schedule)
month_info = get_month_info(schedule)
workers_info = get_workers_info(schedule)

Dict{String,Any} with 1 entry:
  "time" => Dict{String,Any}("nurse_2"=>"FULL","nurse_3"=>"FULL","nurse_1"=>"FU…

In [4]:
get_max_nbhd_size(schedule)

┌ Debug: Neighbors number from addition: 60
└ @ Main.NurseSchedules.Neighborhood ~/Documents/nurse/nurse_scheduling_problem/src/neighborhood.jl:11
┌ Debug: Neighbors number from deletion: 8
└ @ Main.NurseSchedules.Neighborhood ~/Documents/nurse/nurse_scheduling_problem/src/neighborhood.jl:13
┌ Debug: Neighbors number from swap: 17
└ @ Main.NurseSchedules.Neighborhood ~/Documents/nurse/nurse_scheduling_problem/src/neighborhood.jl:26


85

In [5]:
neighborhood = get_nbhd(shifts)
length(neighborhood)

85

In [6]:
schedule_score = score(workers, shifts, month_info, workers_info)

┌ Debug: Lacking nurses on day '1'.
│ Expected '1', got '0' in the day.
│ Expected '1', got '0' at night.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:61
┌ Debug: Lacking nurses on day '3'.
│ Expected '1', got '0' in the day.
│ Expected '1', got '0' at night.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:61
┌ Debug: Lacking nurses on day '4'.
│ Expected '1', got '0' in the day.
│ Expected '1', got '0' at night.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:61
┌ Debug: Lacking nurses on day '5'.
│ Expected '1', got '0' in the day.
│ Expected '1', got '0' at night.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:61
┌ Debug: Lacking nurses on day '7'.
│ Expected '1', got '0' in the day.
│ Expected '1', got '0' at night.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/s

193

In [15]:
global_logger(NullLogger())
ITERATION_NUM = 1000
best_shifts = (shifts, schedule_score)

(["W" "DN" … "W" "W"; "D" "N" … "N" "U"; "R" "R" … "DN" "W"], 193)

In [16]:
iter_best = best_shifts
no_improvement_iters = 0

for i in 1:ITERATION_NUM
    neighborhood = get_nbhd(iter_best[1])
    nghd_scores = map(s -> score(workers, s, month_info, workers_info), neighborhood)
    iter_best_idx = findfirst(nghd_scores .== minimum(nghd_scores))
    new_iter_best = (neighborhood[iter_best_idx], nghd_scores[iter_best_idx])
    
    if best_shifts[2] > iter_best[2]
        println("Improved from $(best_shifts[2]) to $(iter_best[2]) in iteration '$(i)'")
        global best_shifts = iter_best
    end
end

In [17]:
global_logger(logger)
score(workers, best_shifts[1], month_info, workers_info)

┌ Debug: Lacking nurses on day '1'.
│ Expected '1', got '0' in the day.
│ Expected '1', got '0' at night.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:61
┌ Debug: Lacking nurses on day '3'.
│ Expected '1', got '0' in the day.
│ Expected '1', got '0' at night.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:61
┌ Debug: Lacking nurses on day '4'.
│ Expected '1', got '0' in the day.
│ Expected '1', got '0' at night.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:61
┌ Debug: Lacking nurses on day '5'.
│ Expected '1', got '0' in the day.
│ Expected '1', got '0' at night.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:61
┌ Debug: Lacking nurses on day '7'.
│ Expected '1', got '0' in the day.
│ Expected '1', got '0' at night.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/s

193